<a href="https://colab.research.google.com/github/ZhuRuoyu/calendar-chatbot/blob/main/customized_model/FINAL_intents_recognition_fine_tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.linkedin.com/pulse/fine-tuning-your-own-sentence-transformers-python-adie-kaye

In [4]:
%cd /content/drive/MyDrive/capstone_sentence_transformer_output

/content/drive/MyDrive/capstone_sentence_transformer_output


In [1]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer

In [ ]:
# model_pretrained = 'bert-base-nli-mean-tokens'
# path2 = 'sentence-transformers/all-MiniLM-L6-v2'
# path3 = 'sentence-transformers/sentence-t5-large'
# path4 = 'all-mpnet-base-v2'
# original_model = SentenceTransformer(path4)

# path5_finetune = "tuned_models/fine_tuned_model"
# fine_tuned_model = SentenceTransformer(path5_finetune)


model = SentenceTransformer('all-mpnet-base-v2')

In [3]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util
from sentence_transformers.evaluation import TripletEvaluator



# Version 2.1

In [12]:
intents = [
    "create a new event", #0 create event
    "delete a certain event",#1 delete event
    "I want to retrieve event detail", #2 get event detail
    "get free time information during certain time period", #3 get free time
    "change the location of an event",#4 update event (location)
    "change the summary of an event",#5 update event (summary)

    "reschedule the time of an event" #6  reschedule(start time, end time)

    ]

input_sentences = {
"adding a new event":intents[0],
"Insert a fresh event into the schedule.": intents[0],
"Incorporate a novel occasion into the calendar.": intents[0],
"Append a new event to the existing agenda.": intents[0],
"Integrate this recently planned event into the timetable.": intents[0],
"Put in a brand-new event on the calendar.": intents[0],
"Incorporate a new happening into the schedule.": intents[0],
"Register a fresh event on the calendar.": intents[0],
"Include a newly organized event into the agenda.": intents[0],
"Add a recently created event to the calendar.": intents[0],
"Slot a new occasion into the existing schedule.": intents[0],

 "delete the meeting":intents[1],
 "cancel the event today":intents[1],
 "cancel the event":intents[1],
 "delete the event":intents[1],
 "I want to cancel the afternoon events tomorrow":intents[1],
 "clear my schedule for next week.":intents[1], #delete

"What is today's highlighted event?": intents[2],
"Can you tell me the main happening of the day?": intents[2],
"I'd like to know the event that's taking place today, please.": intents[2],
"What is the significant occurrence for today?": intents[2],
"Could you inform me about the event scheduled for today?": intents[2],
"I'm curious about the day's noteworthy event. What is it?": intents[2],
"Can you provide information on the event happening today?": intents[2],
"I'm interested in knowing the special event of the day.": intents[2],
"Could you share details about today's prominent event?": intents[2], #get event detail
"Who is in the meeting?": intents[2],
 "what is the purpose for this 8am meeting?": intents[2],
 "give me the details about the next meeting": intents[2],
 'tell me about the event at 9pm': intents[2], # get event detail

"Am I free for next week?":intents[3],
 "do I have things to do this afternoon?":intents[3],
 "do I have free time between 9-10am?":intents[3],
"what is my availbility ?":intents[3], # get free time information

"Update the event location on the schedule.": intents[4],
"Edit the event details to reflect the new venue.": intents[4],
"Modify the event's location information.": intents[4],
"Alter the venue for the upcoming event.": intents[4],
"Amend the location of the scheduled event.": intents[4],
"Revise the event's location on the calendar.": intents[4],
"Make changes to the event's venue.": intents[4],
"Update the event's location to the new address.": intents[4],
"Adjust the location of the event in the system.": intents[4],
"Switch the event's location to the updated one.": intents[4],

"Update the event summary with the latest information.": intents[5],
"Edit the event description to reflect the changes.": intents[5],
"Modify the summary of the scheduled event.": intents[5],
"Alter the event summary to be more accurate.": intents[5],
"Amend the event summary with the new details.": intents[5],
"Revise the event overview to include additional content.": intents[5],
"Make changes to the event summary accordingly.": intents[5],
"Update the event's summary to match the updates.": intents[5],
"Adjust the event description with the latest changes.": intents[5],
"Modify the summary of the event to reflect recent developments.": intents[5],

"change the 9pm meeting to 10pm.":intents[6],
 "update the meeting time for next Monday":intents[6],

 "can I postpone the post office errand?":intents[6],
 "make the grocery shopping trip earlier":intents[6] # reschedule

}

input_sentences_list = list(input_sentences.keys())


In [ ]:
for i in range(len(input_sentences_list)):
  print('the one training sample is':input_sentences_list[i])

In [ ]:
# intents = [
#     "create a new event", #0 create event
#     "delete a certain event",#1 delete event
#     "I want to retrieve event detail", #2 get event detail
#     "get free time information during certain time period", #3 get free time
#     "change the location of an event",#4 update event (location)
#     "change the summary of an event",#5 update event (summary)

#     "reschedule the time of an event" #6  reschedule(start time, end time)

#     ]

In [19]:
train_example_large = []
for i in range(len(input_sentences_list)):
  anchor_intent_key = input_sentences_list[i]
  positive_intent = input_sentences[anchor_intent_key] # use key to find value 'intents[x]'
  if positive_intent == intents[0]:
    negative_intent = intents[6]
  elif positive_intent == intents[1]:
    negative_intent = intents[5]
  elif positive_intent == intents[2]:
    negative_intent = intents[0]
  elif positive_intent == intents[3]:
    negative_intent = intents[2]
  elif positive_intent == intents[4]:
    negative_intent = intents[5]
  elif positive_intent == intents[5]:
    negative_intent = intents[4]
  elif positive_intent == intents[6]:
    negative_intent = intents[5]

  triple_intent_list = [anchor_intent_key,positive_intent,negative_intent]

  input_new = InputExample(texts=triple_intent_list, label = 1)

  train_example_large.append(input_new)

In [20]:
# Define our training examples
# train_examples = [
#     InputExample(texts=['do I have things to do this afternoon?',intents[3] , intents[5]], label=1),
#     InputExample(texts=['tell me about the event at 9pm', intents[2], intents[5]], label=1),
#     InputExample(texts=['Could you inform me about the event scheduled for today?', intents[2], intents[0]], label=1),
#     InputExample(texts=['Who is in the meeting?', intents[2], intents[4]], label=1),
#     InputExample(texts=['what is the purpose for this 8am meeting?', intents[2], intents[5]], label=1),
#     InputExample(texts=['give me the details about the next meeting', intents[2], intents[5]], label=1),
#     #InputExample(texts=['', intents[], intents[]], label=1),
#     #InputExample(texts=['', intents[], intents[]], label=1),
# ]

### train_example_large

train_dataset = SentencesDataset(train_example_large, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


# Define our evaluator
#evaluator = TripletEvaluator('tell me about the event at 9pm','Retrieve event detail of a certain date.', 'Reschedule the time of an event.')
# Define our evaluator
evaluator = TripletEvaluator(anchors=['Could you inform me about the event scheduled for today?'],
                              positives = ['get free time information during certain time period'],
                              negatives = ['create a new event'])


# Define the loss function
train_loss = losses.TripletLoss(model=model)


# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)


# Save the model
model.save("tuned_models/fine_tuned_model")

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

In [46]:
path5_finetune = "/content/drive/MyDrive/capstone_sentence_transformer_output/tuned_models/fine_tuned_model"
fine_tuned_model = SentenceTransformer(path5_finetune)

# Define our training examples
train_examples_second_fine_tuned = [
    InputExample(texts=['what is the purpose for this 8am meeting?',intents[2] , intents[6]], label=1),
    InputExample(texts=['Modify the summary of the scheduled event.', intents[5], intents[2]], label=1),
    InputExample(texts=['Could you inform me about the event scheduled for today?', intents[6], intents[0]], label=1),
    # InputExample(texts=['Who is in the meeting?', intents[2], intents[4]], label=1),
    # InputExample(texts=['what is the purpose for this 8am meeting?', intents[2], intents[5]], label=1),
    # InputExample(texts=['give me the details about the next meeting', intents[2], intents[5]], label=1),
    #InputExample(texts=['', intents[], intents[]], label=1),
    #InputExample(texts=['', intents[], intents[]], label=1),
]

### train_example_large

train_dataset = SentencesDataset(train_examples_second_fine_tuned, fine_tuned_model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


# Define our evaluator
#evaluator = TripletEvaluator('tell me about the event at 9pm','Retrieve event detail of a certain date.', 'Reschedule the time of an event.')
# Define our evaluator
evaluator = TripletEvaluator(anchors=['Could you inform me about the event scheduled for today?'],
                              positives = ['get free time information during certain time period'],
                              negatives = ['create a new event'])


# Define the loss function
train_loss = losses.TripletLoss(model=model)


# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)


# Save the model
model.save("tuned_models/fine_tuned_model_2")

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

# check the difference between original model VS  fine-tuned model

In [22]:
path4 = 'all-mpnet-base-v2'
original_model = SentenceTransformer(path4)

path5_finetune = "/content/drive/MyDrive/capstone_sentence_transformer_output/tuned_models/fine_tuned_model"
fine_tuned_model = SentenceTransformer(path5_finetune)


In [23]:
sentences = [
    "Could you inform me about the event scheduled for today?",
    "I'm curious about the day's noteworthy event. What is it?",
    "Can you provide information on the event happening today?",
    "I'm interested in knowing the special event of the day.",
    "Could you share details about today's prominent event?"
]

#input_sentences_list

In [24]:

for sentence in input_sentences_list:
    original_embedding = original_model.encode(sentence)
    fine_tuned_embedding = fine_tuned_model.encode(sentence)

    cosine_similarity = util.pytorch_cos_sim(original_embedding, fine_tuned_embedding)
    print(f"The original and fine-tuned embeddings for '{sentence}' have a similarity of {cosine_similarity}")


# run the model on actual new-generated sentences


## testing data

In [28]:
sentences_test = [
    #"create a new event":

    "Add a fresh event to the schedule.",
    "Create an event for next week.",
    "Schedule a new event on the calendar.",

    #"delete a certain event":

    "Remove the specified event from the agenda.",
    "Delete the selected event from the schedule.",
    "Cancel the particular event from the calendar.",


    #"I want to retrieve event detail":

    "Get the details of the chosen event.",
    "Provide me with information about the event.",
    "Retrieve the specifics of the selected event.",


    #"get free time information during certain time period":

    "Show available time slots for the given period.",
    "Get free time details for the specified timeframe.",
    "Retrieve information about vacant slots during a certain time.",

    #"change the location of an event":

    "Update the event venue on the calendar.",
    "Modify the event's location to a new place.",
    "Alter the venue of the scheduled event.",

    #"change the summary of an event":

    "Edit the event description to reflect recent changes.",
    "Modify the summary of the selected event.",
    "Update the event overview with new information.",

    #"reschedule the time of an event":

    "Change the event timing to a different date.",
    "Reschedule the event for another day.",
    "Move the event time to a new schedule.",
]

In [41]:
sentences_test_2 = [
    #"create a new event":

    "Add a new event to the schedule.",
    "Create a fresh event on the calendar.",
    "Register a new event for the upcoming date.",
    "Schedule a brand-new event for the team.",
    "Organize a new event in the system.",
    #"delete a certain event":

    "Remove the specified event from the calendar.",
    "Delete a particular event from the schedule.",
    "Erase the selected event from the system.",
    "Eliminate the specified event entry.",
    "Cancel the chosen event on the calendar.",
    #"I want to retrieve event detail":

    "Get the details of the event requested.",
    "Fetch information about the specific event.",
    "Retrieve the details for the chosen event.",
    "Show me the details of the selected event.",
    "Provide information about the requested event.",
    #"get free time information during a certain time period":

    "Show available time slots during the specified period.",
    "Get information about free time within the chosen timeframe.",
    "Display available time blocks for the given duration.",
    "Retrieve details of vacant time slots in the selected period.",
    "Provide me with free time information within the specified time range.",
    #"change the location of an event":

    "Update the event location on the schedule.",
    "Edit the event details to reflect the new venue.",
    "Modify the location of the scheduled event.",
    "Alter the venue for the upcoming event.",
    "Amend the location of the chosen event.",
    #"change the summary of an event":

    "Update the event summary with the latest information.",
    "Edit the event description to reflect the changes.",
    "Modify the summary of the scheduled event.",
    "Revise the event overview with additional content.",
    "Adjust the event summary to match the updates.",
    #"reschedule the time of an event":

    "Change the event time on the calendar.",
    "Update the schedule for the chosen event.",
    "Modify the time of the scheduled event.",
    "Adjust the event time to a new slot.",
    "Move the event to a different time.",
    ]

In [45]:
sentences_test_3 = [
    # similar to "what is the purpose for this 8am meeting?"
    "Could you clarify the purpose of the 8am meeting?",
    "I'm curious about the objective behind the 8am meeting.",
    "Can you provide some insights into the reason for the 8am meeting?",

    # similar to "Modify the summary of the scheduled event."
    "Revise the event summary for the upcoming occasion.",
    "Edit the summary of the planned event in the calendar.",
    "Adjust the summary of the scheduled event to include recent updates.",

    # similar to "Update the schedule for the chosen event."
    "Revise the timetable for this 8am event.",
    "Modify the time for the pizza party tonight.",
    "Adjust the schedule for the 9pm meeting."

]

## embed the testing data

In [30]:
intents_embed = fine_tuned_model.encode(intents)
input_embed = fine_tuned_model.encode(sentences_test) #testing data

print(intents_embed.shape) #(5, 768)
print(input_embed.shape)

(7, 768)
(21, 768)


In [33]:
input_embed_training_data = fine_tuned_model.encode(input_sentences_list) #training data
print(input_embed_training_data.shape)

(58, 768)


In [42]:
input_embed_2 = fine_tuned_model.encode(sentences_test_2)

print(input_embed_2.shape)

(35, 768)


In [48]:
path6_finetune = "/content/drive/MyDrive/capstone_sentence_transformer_output/tuned_models/fine_tuned_model_2"
fine_tuned_model_2 = SentenceTransformer(path6_finetune)


In [49]:
input_embed_3 = fine_tuned_model_2.encode(sentences_test_3)

print(input_embed_3.shape)

(9, 768)


In [51]:
input_embed_4 = fine_tuned_model_2.encode(sentences_test_2)

print(input_embed_4.shape)

(35, 768)


## run cosine similarity and check the result

In [31]:

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [52]:
for i in range(len(input_embed_4)):
  print('INPUT SENTENCE: ',sentences_test_2[i])
  vector = input_embed_4[i]
  cosine_score = cosine_similarity([vector], intents_embed)
  #print(cosine_score)
  index = np.argmax(cosine_score, axis = 1)[0]
  print(index)
  print('INTENTS SENTENCE:',intents[index])
  print('\n')
  print('-------- ----------')

INPUT SENTENCE:  Add a new event to the schedule.
0
INTENTS SENTENCE: create a new event


-------- ----------
INPUT SENTENCE:  Create a fresh event on the calendar.
0
INTENTS SENTENCE: create a new event


-------- ----------
INPUT SENTENCE:  Register a new event for the upcoming date.
0
INTENTS SENTENCE: create a new event


-------- ----------
INPUT SENTENCE:  Schedule a brand-new event for the team.
0
INTENTS SENTENCE: create a new event


-------- ----------
INPUT SENTENCE:  Organize a new event in the system.
0
INTENTS SENTENCE: create a new event


-------- ----------
INPUT SENTENCE:  Remove the specified event from the calendar.
1
INTENTS SENTENCE: delete a certain event


-------- ----------
INPUT SENTENCE:  Delete a particular event from the schedule.
1
INTENTS SENTENCE: delete a certain event


-------- ----------
INPUT SENTENCE:  Erase the selected event from the system.
1
INTENTS SENTENCE: delete a certain event


-------- ----------
INPUT SENTENCE:  Eliminate the specifie

## <font color="red">yay the model works! </font>

# -------------------------ignore below---------------------------
# IGNORE - Version 1

In [4]:
# def triplet_loss(anchor, positive, negative, margin=0.2):
#     """
#     anchor, positive, negative are 1-D tensors (embeddings)
#     margin is a hyperparameter
#     """
#     pos_distance = torch.norm(anchor - positive, p=2)
#     neg_distance = torch.norm(anchor - negative, p=2)

#     basic_loss = pos_distance - neg_distance + margin
#     loss = torch.clamp(basic_loss, min=0.0)

#     return loss

In [17]:


# model = SentenceTransformer('all-mpnet-base-v2')

# # Define our training examples
# train_examples = [
#     InputExample(texts=['I can\'t log in to my account.', 'Unable to access my account.', 'I need help with the payment process.'], label=1),
#     InputExample(texts=['How do I reset my password?', 'I forgot my password. What do I do?', 'How can I upgrade my account?'], label=1),
#     InputExample(texts=['My order hasn\'t arrived yet.', 'I haven\'t received my package.', 'I can\'t find the logout button.'], label=1),
#     # More examples... like, a LOT more examples (you'll find out why soon)
# ]
# train_dataset = SentencesDataset(train_examples, model)
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


# # Define our evaluator
# evaluator = TripletEvaluator(anchors=['How can I change my password?'],
#                               positives = ['What is the process for password change?'],
#                               negatives = ['My package is delayed.'])
# #evaluator = TripletEvaluator()

# # Define the loss function
# train_loss = losses.TripletLoss(model=model)


# # Fine-tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)


# # Save the model
# model.save("tuned_models/fine_tuned_model")

In [19]:
# intents = [
#     "Create a new event.", #0
#     "Delete a certain event.",#1
#     "Retrieve event detail of a certain date.", #2 -> TAG: retrieving event (intent) today (entity arguments)
#     "Get free time information during certain time period.", #3
#     "Change the location or summary of an event.",#4
#     "Reschedule the time of an event." #5
#     ]

In [ ]:
# for i in range(6):
#   print(i, intents[i])


0 Create a new event.
1 Delete a certain event.
2 Retrieve event detail of a certain date.
3 Get free time information during certain time period.
4 Change the location or summary of an event.
5 Reschedule the time of an event.


In [ ]:
# # Define our training examples
# train_examples = [
#     InputExample(texts=['do I have things to do this afternoon?',intents[3] , intents[5]], label=1),
#     InputExample(texts=['tell me about the event at 9pm', intents[2], intents[5]], label=1),
#     InputExample(texts=['Could you inform me about the event scheduled for today?', intents[2], intents[0]], label=1),
#     InputExample(texts=['Who is in the meeting?', intents[2], intents[4]], label=1),
#     InputExample(texts=['what is the purpose for this 8am meeting?', intents[2], intents[5]], label=1),
#     InputExample(texts=['give me the details about the next meeting', intents[2], intents[5]], label=1),
#     #InputExample(texts=['', intents[], intents[]], label=1),
#     #InputExample(texts=['', intents[], intents[]], label=1),
# ]
# train_dataset = SentencesDataset(train_examples, model)
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


# # Define our evaluator
# #evaluator = TripletEvaluator('tell me about the event at 9pm','Retrieve event detail of a certain date.', 'Reschedule the time of an event.')
# # Define our evaluator
# evaluator = TripletEvaluator(anchors=['tell me about the event at 9pm'],
#                               positives = ['Retrieve event detail of a certain date.'],
#                               negatives = ['Reschedule the time of an event.'])


# # Define the loss function
# train_loss = losses.TripletLoss(model=model)


# # Fine-tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=40, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)


# # Save the model
# model.save("tuned_models/fine_tuned_model")